## Import .msh file in fenics

In [72]:
# import meshio
# mesh_from_file = meshio.read("path/to/msh/file")

# import numpy
# def create_mesh(mesh, cell_type, prune_z=False):
#     cells = mesh.get_cells_type(cell_type)
#     points = mesh.points[:,:2] if prune_z else mesh.points
#     out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
#     return out_mesh

# triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)
# meshio.write("path/to/xdmf/file", triangle_mesh)

# from dolfin import * 
# mesh = Mesh()
# with XDMFFile("path/to/xdmf/file") as infile:
#     infile.read(mesh)
    
# plot(mesh);

## Extract cell midpoints from msh file and add them to dat file

In [73]:
# import meshio
# import numpy as np

# def convert_msh_to_dat(msh_file, dat_file):
#     # Read the mesh file
#     mesh = meshio.read(msh_file)

#     # Get the cell midpoints
#     cell_midpoints = []
#     for cell_block in mesh.cells:
#         cell_type = cell_block.type
#         cell_data = cell_block.data
#         if cell_type == "triangle":
#             for cell_vertices in cell_data:
#                 # Calculate the midpoint of the cell
#                 midpoint = np.mean(mesh.points[cell_vertices], axis=0)
#                 cell_midpoints.append(midpoint)

#     # Convert the cell midpoints to a numpy array
#     cell_midpoints = np.array(cell_midpoints)

#     # Save the cell midpoints to the .dat file
#     np.savetxt(dat_file, cell_midpoints, fmt="%.6f")

# # Specify the input .msh file and the output .dat file
# msh_file = "path/to/msh/file"
# dat_file = "path/to/dat/file"

# # Convert .msh to .dat
# convert_msh_to_dat(msh_file, dat_file)


## Flip the micrograph

In [74]:
# from PIL import Image, ImageOps
# import matplotlib.pyplot as plt

# def convert_colors(image_path):
#     # Load the image
#     image = Image.open(image_path)
#     image = image.convert("RGB")  # Convert image to RGB mode
#     pixels = image.load()

#     width, height = image.size

#     # Flip the modified image vertically
#     modified_image = ImageOps.flip(image)

#     # Save the modified image
#     new_image_path = "modified_image.jpg"  # Specify the desired output file path and name
#     modified_image.save(new_image_path)

#     print("Image flipping completed. Modified image saved as", new_image_path)

#     # Display the modified image
#     plt.imshow(modified_image)
#     plt.axis('off')
#     plt.show()

# def in_range(color, color_range):
#     # Check if the color is within the specified range
#     min_values, max_values = color_range
#     return all(c >= min_v and c <= max_v for c, (min_v, max_v) in zip(color, zip(min_values, max_values)))

# # Example usage
# image_path = "path/to/image/to/flip"  # Specify the path to your input image file
# convert_colors(image_path)


## Add image, msh file get dat file with fourth colmn as material property 
## CAUTION: ALWAYS ADD THE FLIPPED BOY HERE (FLIPPED AROUND Y = IMAGE HEIGHT HERE)

In [84]:
# import numpy as np
# import meshio
# import imageio

# def convert_image_to_dat(image_file, msh_file, dat_file):
#     # Load the image
#     image = imageio.v2.imread(image_file)

#     # Load the mesh from .msh file
#     mesh = meshio.read(msh_file)

#     # Get the cell midpoints
#     cell_midpoints = []
#     for cell_block in mesh.cells:
#         cell_type = cell_block.type
#         cell_data = cell_block.data
#         if cell_type == "triangle":
#             for cell_vertices in cell_data:
#                 # Calculate the midpoint of the cell
#                 midpoint = np.mean(mesh.points[cell_vertices], axis=0)
#                 cell_midpoints.append(midpoint)

#     # Convert the image to .dat
#     with open(dat_file, "w") as file:
#         for midpoint in cell_midpoints:
#             # Determine the color of the cell
#             x_index = np.clip(int(midpoint[0]), 0, image.shape[1] - 1)
#             y_index = np.clip(int(midpoint[1]), 0, image.shape[0] - 1)
#             color = image[y_index, x_index][:3]

#             # Determine the fourth column value based on color
#             fourth_column = 1 if color[0] >= 127.5 else 0

#             # Write the XYZ coordinates and fourth column value to the .dat file
#             file.write(f"{midpoint[0]} {midpoint[1]} {midpoint[2]} {fourth_column}\n")


# # Specify the input image file, the mesh file (.msh), and the output .dat file
# image_file = "path/to/flip/image"
# msh_file = "path/to/msh/file"
# dat_file = "path/to/dat/file"

# # Convert the image to .dat
# convert_image_to_dat(image_file, msh_file, dat_file)

## Extract material property and store in 1d array and see material id 

In [76]:
# def extract_fourth_column(dat_file):
#     fourth_column = []
#     with open(dat_file, "r") as file:
#         for line in file:
#             elements = line.strip().split()
#             fourth_column.append(int(elements[3]))  # Assuming the fourth column is at index 3
#     return fourth_column

# # Provide the path to the DAT file
# dat_file = "path/to/dat/file"

# # Extract the fourth column elements
# fourth_column_array = extract_fourth_column(dat_file)
# fourth_column_array

## Write that 1d array into xdmf file in mesh cell midpoint 

In [77]:
# import dolfin

# materials = MeshFunction('double', mesh, 2)

# V = VectorFunctionSpace(mesh, "Lagrange", 2)
# u = Function(V)

# local_values_material = np.zeros_like(u.vector().get_local())

# for cell in cells(mesh):
#     midpoint = cell.midpoint().array()
#     i = (midpoint[0])
#     j = (midpoint[1])
#     k = (midpoint[2])
#     local_values_material[cell.index()] = fourth_column_array[cell.index()]
#     materials[cell] = int(local_values_material[cell.index()])
#     print(midpoint, i, j , k, "Material:", materials[cell], "Cell index: ", cell.index())
    
# u.vector().set_local(local_values_material)

# dolfin.XDMFFile(dolfin.MPI.comm_world, "path/to/xdmf/file").write_checkpoint(u,"u",0)

## Assigning different material properties to cell with different either material tag 0 or 1

In [78]:
# import dolfin as dol
# class al(dol.UserExpression):
#     def __init__(self, materials, al0, al1, **kwargs):
#         super().__init__(**kwargs)
#         self.materials = materials
#         self.k_0 = al0
#         self.k_1 = al1
#     def eval_cell(self, values, x, cell):
#         if self.materials[cell.index] == 0:
#             values[0] = self.k_0
#         else:
#             values[0] = self.k_1
            
# E1 = 21e3 (matrix)
# nu1 = 0.3
# E2 = 2100e3 (inclusion)
# nu2 = 0.25

# E = al(materials, E1, E2, degree = 0)
# nu = al(materials, nu1, nu2, degree = 0)

# File('path/to/save/.pvd/file') << project(E, FunctionSpace(mesh, 'DG', 0))
# File('path/to/save/.pvd/file') << materials

## Defining boundary conditions

In [79]:
# class left(SubDomain):
#     def inside(self, x, on_boundary):
#         return on_boundary and near(x[0], 0.)

# class right(SubDomain):
#     def inside(self, x, on_boundary):
#         return on_boundary and near(x[0], 1023.)


# boundaries = MeshFunction("size_t", mesh, mesh.geometry().dim()-1)

# left().mark(boundaries, 1) 
# right().mark(boundaries, 2) 

# bcs = [DirichletBC(V, Constant((0., 0.)), boundaries, 1), DirichletBC(V.sub(0), Constant((0.002)), boundaries, 2)]


## Constitutive relations and constants

In [80]:
# def eps(v):
#     return sym(grad(v))

# def sigma(v):
#     return lmbda*tr(eps(v))*Identity(2) + 2.0*mu*eps(v)

# mu = E/2/(1+nu)
# lmbda = E*nu/(1+nu)/(1-2*nu)

## Variational form, pde solve, plot of displacement

In [81]:
# f = 0.
# f = Constant((0, f))

# du = TrialFunction(V)
# u_ = TestFunction(V)
# a = inner(sigma(du), eps(u_))*dx
# l = inner(f, u_)*dx

# u = Function(V, name="Displacement")
# solve(a == l, u, bcs)

# plot(1e3*u, mode="displacement")

## Projecting Stress and storing in xdmf to visualize in paraview

In [82]:
# Vsig = TensorFunctionSpace(mesh, "DG", degree=0)
# sig = Function(Vsig, name="Stress")
# sig.assign(project(sigma(u), Vsig))

In [83]:
# file_results = XDMFFile("path/to/save/.xdmf/file")
# file_results.parameters["flush_output"] = True
# file_results.parameters["functions_share_mesh"] = True
# file_results.write(u, 0.)
# file_results.write(sig, 0.)